In [5]:
import os

# Path to your dataset
dataset_path = r"E:\CLD_project\Final_CLD_data"

# Subdirectories for train, val, and test
splits = ['train', 'val', 'test']

# Dictionary to store image counts
image_counts = {}

for split in splits:
    split_path = os.path.join(dataset_path, split)
    if os.path.exists(split_path):
        image_counts[split] = {}
        for cls in os.listdir(split_path):
            class_folder = os.path.join(split_path, cls)
            if os.path.isdir(class_folder):
                image_counts[split][cls] = len([f for f in os.listdir(class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

# Print results
for split, classes in image_counts.items():
    print(f"\n{split.upper()} SET:")
    for cls, count in classes.items():
        print(f"  Class '{cls}': {count} images")



TRAIN SET:
  Class 'Cerscospora': 25000 images
  Class 'Healthy': 24999 images
  Class 'Leaf rust': 25000 images
  Class 'Miner': 25000 images
  Class 'Phoma': 25000 images

VAL SET:
  Class 'Cerscospora': 5000 images
  Class 'Healthy': 5000 images
  Class 'Leaf rust': 5000 images
  Class 'Miner': 5000 images
  Class 'Phoma': 5000 images

TEST SET:
  Class 'Cerscospora': 769 images
  Class 'Healthy': 1899 images
  Class 'Leaf rust': 835 images
  Class 'Miner': 1699 images
  Class 'Phoma': 658 images


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader
from torchvision.models import ResNet50_Weights

import matplotlib.pyplot as plt
import os
from tqdm import tqdm

# ✅ Set Paths
DATASET_PATH = "E:\CLD_project\Final_CLD_data"
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
VAL_PATH = os.path.join(DATASET_PATH, "val")
TEST_PATH = os.path.join(DATASET_PATH, "test")

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 3

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


In [2]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_dataset = datasets.ImageFolder(TRAIN_PATH, transform=transform)
val_dataset = datasets.ImageFolder(VAL_PATH, transform=transform)
test_dataset = datasets.ImageFolder(TEST_PATH, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Train: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")


Train: 124999, Validation: 25000, Test: 5860


In [3]:
# ✅ Load Pretrained ResNet50
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_dataset.classes))  # Adjust output layer

model = model.to(DEVICE)

In [4]:
# ✅ Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Lists to store metrics
train_losses, val_losses = [], []
train_accs, val_accs = [], []
